# Example illustrating different forecast evaluation statistics for precipitation

## This is part 2; expects that data have already been preprocessed using part 1, with a netcdf file written out

## We'll use the heavy rainfall in eastern Colorado from 21-22 June 2023 for the example

[![Latest release](https://badgen.net/github/release/Naereen/Strapdown.js)](https://github.com/eabarnes1010/ai_weather_to_climate_ats780A8/tree/main/lecture_code)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eabarnes1010/ai_weather_to_climate_ats780A8/blob/main/lecture_code/precip_verif_example.ipynb)

In [ ]:
!pip install cartopy metpy plotly netcdf4

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import fsspec

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)

import metpy.calc as mpcalc
import plotly.express as px

In [ ]:
vtime = pd.Timestamp(2023,6,22,12) ### valid time

fxx = 24 ### what lead time do we want to evaluate? (in hours)

init = vtime - pd.Timedelta(hours=fxx)   ### this will give us the forecasts from 12 UTC 21 June (a 24-h forecast)
init


## Read in the netcdf we wrote in the other part. Need to use fsspec because xarray doesn't support this directly (see https://github.com/pydata/xarray/issues/3653) 

In [ ]:
url = "https://github.com/eabarnes1010/ai_weather_to_climate_ats780A8/raw/main/lecture_code/precip_data_preproc_2023062212.nc#mode=bytes"

with fsspec.open(url) as fobj:
    data_all = xr.open_dataset(fobj)

data_all

### subset data spatially

In [ ]:
### set up some borders to subset the data
minlon = -108.
maxlon = -101.
minlat = 36.
maxlat = 42.

data_sub = data_all.sel(lon=slice(minlon-2,maxlon+2), lat=slice(minlat-2,maxlat+2))
data_sub

### set up the color table and map background

In [ ]:
## set up NWS color table
cmap_data = [(1.000000, 1.000000, 1.000000),
(0.498039, 1.000000, 0.000000),
(0.000000, 0.803922, 0.000000),
(0.000000, 0.545098, 0.000000),
(0.062745, 0.305882, 0.545098),
(0.117647, 0.564706, 1.000000),
(0.000000, 0.698039, 0.933333),
(0.000000, 0.933333, 0.933333),
(0.537255, 0.407843, 0.803922),
(0.568627, 0.172549, 0.933333),
(0.545098, 0.000000, 0.545098),
(0.545098, 0.000000, 0.000000),
(0.803922, 0.000000, 0.000000),
(0.933333, 0.250980, 0.000000),
(1.000000, 0.498039, 0.000000),
(0.803922, 0.521569, 0.000000),
(1.000000, 0.843137, 0.000000)]
cmap = mcolors.ListedColormap(cmap_data, 'precip_rss')

#clevs = [0,0.01,0.1,0.25,0.5,0.75,1.00,1.5,2,2.5,3,3.5,4,5,6,8,10]  ### inches
clevs = [0,0.25,2.5,5,10,20,25,37.5,50,62.5,75,87.5,100,125,150,200,250]  ## mm
norm = mcolors.BoundaryNorm(clevs, cmap.N)


def plot_background(ax,minlon,maxlon,minlat,maxlat):
    ax.set_extent([minlon,maxlon,minlat,maxlat])
    #ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.5)
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=1)
    ax.add_feature(cfeature.BORDERS.with_scale('10m'), linewidth=1)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.left_labels = True
    gl.bottom_labels = True
    gl.right_labels = False
    gl.xlines = False
    gl.ylines = False
    gl.x_inline = False
    gl.y_inline = False
    gl.ylocator = LatitudeLocator()
    gl.xformatter = LongitudeFormatter()
    gl.yformatter = LatitudeFormatter()
    gl.rotate_labels = False
    gl.xlabel_style = {'size': 7}
    gl.ylabel_style = {'size': 7}

    return ax

In [ ]:
crs = ccrs.LambertConformal(central_longitude=-105., central_latitude=40.)
lon2d,lat2d = np.meshgrid(data_sub.lon,data_sub.lat)

datasets_to_plot = ['stage4','era5','gfs','hrrr','hrrr_smooth','hrrr_smooth']

fig, ax_dict = plt.subplot_mosaic([datasets_to_plot[0:2],
                                   datasets_to_plot[2:4],
                                  datasets_to_plot[4:6]],
                              layout='constrained',
                               figsize=(8,10),
                               subplot_kw={'projection': crs})

axlist = []
for k, ax in ax_dict.items():
    axlist=axlist+[ax]
    plot_background(ax,minlon,maxlon,minlat,maxlat)

for ds in datasets_to_plot:
    cf = ax_dict[ds].contourf(lon2d,lat2d,data_sub[ds],
                         clevs, cmap=cmap, norm=norm, 
                              extend='max',
                         transform_first=True,
                         transform=ccrs.PlateCarree())

plt.suptitle("precipitation analysis/forecast comparison for 24 h ending "+str(vtime), 
            fontweight='semibold')

cb = plt.colorbar(cf, ax=axlist, orientation='vertical', 
                  ticks=clevs, norm=norm, aspect=50, shrink=0.85, pad=0.01)
cb.ax.tick_params(labelsize=10)
cb.set_label('precipitation (mm)', fontsize=10)

### label panels, from here: 
### https://matplotlib.org/stable/gallery/text_labels_and_annotations/label_subplots.html#sphx-glr-gallery-text-labels-and-annotations-label-subplots-py
for label, ax in ax_dict.items():
    ax.annotate(
        label,
        xy=(0, 1), xycoords='axes fraction',
        xytext=(+0.5, -0.5), textcoords='offset fontsize',
        fontsize=12, fontweight='semibold', verticalalignment='top',
        bbox=dict(facecolor='white', edgecolor='black', pad=3.0))

plt.show()
    


## OK, now let's actually calculate some verification stats! We will use the 'scores' package which makes this fairly straightforward for a wide variety of scores

In [ ]:
import operator
import scores.categorical

### first we can use a single rainfall threshold, and inspect the contingency table for a forecast
#### Building off of the example here: https://scores.readthedocs.io/en/stable/tutorials/Binary_Contingency_Scores.html

In [ ]:
threshold = 0.25  ### 0.25 mm = 0.01 inch which is generally used as 'measureable' rain

event_operator = scores.categorical.ThresholdEventOperator(default_event_threshold=threshold, default_op_fn=operator.ge)

### GFS forecast, compared to 'truth' of Stage IV
c_man = event_operator.make_contingency_manager(data_sub['gfs'], data_sub['stage4'])

c_man.get_table()  ### the 5 entries in the array are: true positive, true negative, false positive, false negative, and total


#### now we can see some of the scores that can be computed from the contingency table

In [ ]:
print("GFS statistics:")
print("POD/recall = "+str(c_man.probability_of_detection().values))
print("success ratio/precision = "+str(c_man.success_ratio().values))
print("critical success index = "+str(c_man.critical_success_index().values))
print("equitable threat score = "+str(c_man.equitable_threat_score().values))
print("F1 score = "+str(c_man.f1_score().values))


### let's now loop over multiple rainfall thresholds and calculate the scores for those. We'll compile everything into a pandas dataframe

In [ ]:
first_time = True  ### for first time through loop

for threshold in [0.25,2.5,5,10,25,35,50]:
    print("threshold: "+str(threshold)+" mm")

    event_operator = scores.categorical.ThresholdEventOperator(default_event_threshold=threshold, 
                                                               default_op_fn=operator.ge)

    first_ds = True
    for ds in datasets_to_plot[1:-1]:
        c_man = event_operator.make_contingency_manager(data_sub[ds], data_sub['stage4'])

        scores_df_this = pd.DataFrame([c_man.probability_of_detection().values,
                          c_man.success_ratio().values,
                          c_man.critical_success_index().values,
                          c_man.equitable_threat_score().values,
                          c_man.f1_score().values,
                          c_man.heidke_skill_score().values], columns=[ds])

        scores_df_this['score'] = ["POD","SR","CSI","ETS","F1","HSS"] ### list of scores shown above
        scores_df_this['threshold'] = threshold

        if first_ds:
            scores_df = scores_df_this
            first_ds = False
        else:
            scores_df = pd.concat([scores_df, scores_df_this[ds]], axis=1)

    if first_time:
        scores_df_all = scores_df
        first_time = False
    else:
        scores_df_all = pd.concat([scores_df_all, scores_df])

scores_df_all.set_index(['score','threshold'], inplace=True)
scores_df_all
    


### and now we can make some plots of these scores

#### ETS

In [ ]:
scores_df_all.xs('ETS', level='score')

In [ ]:
scores_df_all.xs('ETS', level='score').plot(marker='o')

plt.title("Equitable threat score")

In [ ]:
px.line(scores_df_all.xs('ETS', level='score'),
       title='Equitable threat score', markers=True)

#### F1

In [ ]:
scores_df_all.xs('F1', level='score').plot(marker='o')

plt.title("F1 score")

In [ ]:
px.line(scores_df_all.xs('F1', level='score'),
       title='F1 score', markers=True)

#### Heidke skill score

In [ ]:
scores_df_all.xs('HSS', level='score').plot(marker='o')

plt.title("Heidke skill score")

In [ ]:
px.line(scores_df_all.xs('HSS', level='score'),
       title='Heidke threat score', markers=True)

## Performance diagram (Roebber 2009)
#### Code adapted from DJ Gagne: https://gist.github.com/djgagne/64516e3ea268ec31fb34


In [ ]:
legend_params=dict(loc=4, fontsize=8, framealpha=1, frameon=True)
markers=['*','o','s','v','8']
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']


plt.figure(figsize=(9,9))

grid_ticks = np.arange(0, 1.01, 0.01)
sr_g, pod_g = np.meshgrid(grid_ticks, grid_ticks)
bias = pod_g / sr_g
csi = 1.0 / (1.0 / sr_g + 1.0 / pod_g - 1.0)
csi_contour = plt.contourf(sr_g, pod_g, csi, np.arange(0.1, 1.1, 0.1), extend="max", cmap="Blues")
b_contour = plt.contour(sr_g, pod_g, bias, [0.5, 1, 1.5, 2, 4], colors="k", linestyles="dashed")
plt.clabel(b_contour, fmt="%1.1f", manual=[(0.2, 0.9), (0.4, 0.9), (0.6, 0.9), (0.7, 0.7)])

#### plot our data here:
d=0
for ds in scores_df_all.columns:
    c = 0
    for thresh in scores_df_all.xs('POD', level='score').index:
        plt.plot(scores_df_all.xs('SR', level='score')[ds][thresh],
                 scores_df_all.xs('POD', level='score')[ds][thresh],
                marker=markers[d], markersize=10, markerfacecolor='none',
                 markeredgecolor=colors[c], color=colors[c], markeredgewidth=2,
                 label=ds+", "+str(thresh))
        c=c+1
    d=d+1
                 
cbar = plt.colorbar(csi_contour, pad=0.02, shrink=0.95, aspect=50)
cbar.set_label("Critical Success Index", fontsize=12)
plt.xlabel("Success Ratio (1-FAR)", fontsize=12)
plt.ylabel("Probability of Detection", fontsize=12)
ticks=np.arange(0, 1.1, 0.1)
plt.xticks(ticks)
plt.yticks(ticks)
plt.title("Performance diagram", fontsize=13, fontweight="bold")
#plt.text(0.48,0.6,"Frequency Bias",fontdict=dict(fontsize=14, rotation=45))
plt.legend(**legend_params)

#plt.savefig(filename, dpi=dpi, bbox_inches="tight")

## Synthesis and possible next steps

- What did you take away from this analysis?
- What happened when we smoothed out the HRRR forecast?
- How do the results change if you use ERA5 as the 'truth' instead of Stage-IV?
- What's not being captured by these statistics?